<a href="https://colab.research.google.com/github/hoangviet2/My-Chess-Engine/blob/Ben/ChessEngineModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install maia2


In [ ]:
pip install chess einops gdown numpy pandas pyzstd Requests torch tqdm pyyaml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 23.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.6/413.6 kB 25.3 MB/s eta 0:00:00
  Created wheel for chess: filename=chess-1.11.2-py3-none-any.whl size=147775 sha256=8452dccec30b2681e6e765b554f28dc9376ccf8512d3335c5249bc7eb4e22c65
  Stored in directory: /root/.cache/pip/wheels/83/1f/4e/8f4300f7dd554eb8de70ddfed96e94d3d030ace10c5b53d447
Successfully built chess


In [ ]:
from maia2 import model, dataset, inference

In [ ]:
data = dataset.load_example_test_dataset()

Downloading...
From: https://drive.google.com/uc?id=1fSu4Yp8uYj7xocbHAbjBP6DthsgiJy9X
To: /content/maia2_data/example_test_dataset.csv
100%|██████████| 22.5M/22.5M [00:00<00:00, 69.2MB/s]


Example test dataset downloaded.


In [ ]:
maia2_model = model.from_pretrained(type="rapid", device="gpu")

Model for rapid games already downloaded.
Model for rapid games loaded to gpu.


In [ ]:
data, acc = inference.inference_batch(data, maia2_model, verbose=1, batch_size=1024, num_workers=4)
print(acc)

100%|██████████| 105/105 [01:18<00:00,  1.33it/s]


0.5311


In [ ]:
from maia2 import train, utils

In [ ]:
import requests
import re

def download_all_games(username: str, output_file: str = "all_games.pgn"):
    headers = {"User-Agent": "MyChessDownloader/1.0"}

    # Step 1: Get archives
    archives_url = f"https://api.chess.com/pub/player/{username}/games/archives"
    response = requests.get(archives_url, headers=headers)

    if response.status_code != 200:
        raise Exception(f"Error fetching archives for {username} (HTTP {response.status_code})")

    archives = response.json().get("archives", [])

    if not archives:
        print("⚠️ No archives found for this user.")
        return

    # Step 2: Fetch PGNs
    with open(output_file, "w", encoding="utf-8") as outfile:
        for url in archives:
            pgn_url = f"{url}/pgn"
            print(f"Fetching {pgn_url} ...")

            pgn_response = requests.get(pgn_url, headers=headers)
            if pgn_response.status_code == 200:
                pgn_text = pgn_response.text
                pgn_text = re.sub(r'(\[%clk \d+:\d+:\d+)\.\d+\]', r'\1]', pgn_text)


            # 🔥 Add "Rapid" to Event line so it passes your filter
                games = pgn_text.strip().split("\n\n\n")  # Chess.com separates games with triple newlines
                for g in games:
                    if '[Variant "' in g:  # ❌ skip non-standard variants
                        continue
                    outfile.write(g.strip() + "\n\n\n")
                pgn_text = pgn_text.replace('[Event "Live Chess"]',
                                        '[Event "Live Chess Rated Rapid"]')
                #outfile.write(pgn_response.text + "\n\n")
            else:
                print(f"⚠️ Skipped {pgn_url} (HTTP {pgn_response.status_code})")

    print(f"✅ All games saved to {output_file}")


In [ ]:
download_all_games("sukhoi35","lichess_db_standard_rated_2013-01.pgn")

Fetching https://api.chess.com/pub/player/sukhoi35/games/2016/05/pgn ...
Fetching https://api.chess.com/pub/player/sukhoi35/games/2016/06/pgn ...
Fetching https://api.chess.com/pub/player/sukhoi35/games/2016/07/pgn ...
Fetching https://api.chess.com/pub/player/sukhoi35/games/2016/08/pgn ...
Fetching https://api.chess.com/pub/player/sukhoi35/games/2016/09/pgn ...
Fetching https://api.chess.com/pub/player/sukhoi35/games/2016/10/pgn ...
Fetching https://api.chess.com/pub/player/sukhoi35/games/2016/11/pgn ...
Fetching https://api.chess.com/pub/player/sukhoi35/games/2016/12/pgn ...
Fetching https://api.chess.com/pub/player/sukhoi35/games/2017/01/pgn ...
Fetching https://api.chess.com/pub/player/sukhoi35/games/2017/02/pgn ...
Fetching https://api.chess.com/pub/player/sukhoi35/games/2017/03/pgn ...
Fetching https://api.chess.com/pub/player/sukhoi35/games/2017/04/pgn ...
Fetching https://api.chess.com/pub/player/sukhoi35/games/2017/05/pgn ...
Fetching https://api.chess.com/pub/player/sukhoi35/

In [ ]:
import chess.pgn

def validate_pgn(file_path):
    with open(file_path, encoding="utf-8") as f:
        game_num = 1
        while True:
            try:
                game = chess.pgn.read_game(f)
                if game is None:
                    break
                game_num += 1
            except Exception as e:
                print(f"❌ Error parsing game {game_num}: {e}")
                break


In [ ]:
validate_pgn("/content/all_games.pgn")

In [ ]:
pip install zstandard

In [ ]:
import zstandard as zstd

def compress_pgn(input_file: str, output_file: str):
    """
    Compress a PGN file into .pgn.zst format using Zstandard.
    """
    with open(input_file, "rb") as f_in, open(output_file, "wb") as f_out:
        cctx = zstd.ZstdCompressor(level=3)  # level 1-22 (higher = more compression, slower)
        f_out.write(cctx.compress(f_in.read()))

    print(f"✅ Compressed {input_file} -> {output_file}")

In [ ]:
compress_pgn("lichess_db_standard_rated_2013-01.pgn","lichess_db_standard_rated_2013-01.pgn.zst")

✅ Compressed lichess_db_standard_rated_2013-01.pgn -> lichess_db_standard_rated_2013-01.pgn.zst


In [ ]:
from multiprocessing import Process, Queue, cpu_count
cfg = utils.parse_args(cfg_file_path="/content/config.yaml")
pgn_chunks = utils.read_or_create_chunks("/content/lichess_db_standard_rated_2013-01.pgn", cfg)
print("Number of chunks:", len(pgn_chunks))
pgn_chunks_sublists = []
num_processes = cpu_count() - cfg.num_cpu_left
for i in range(0, len(pgn_chunks), num_processes):
                pgn_chunks_sublists.append(pgn_chunks[i:i + num_processes])
print(pgn_chunks_sublists[0])

Loading cached chunks from /content/lichess_db_standard_rated_2013-01_chunks.pkl
Number of chunks: 12134
[(0, 6844), (6844, 14776)]


In [9]:
cfg = utils.parse_args(cfg_file_path="/content/drive/MyDrive/ChessEngine/config.yaml")
train.run(cfg)

Configurations:
	data_root: /content/drive/MyDrive/ChessEngine
	seed: 42
	num_workers: 16
	verbose: 1
	max_epochs: 3
	max_ply: 300
	clock_threshold: 30
	chunk_size: 250
	start_year: 2025
	start_month: 1
	end_year: 2025
	end_month: 6
	from_checkpoint: False
	checkpoint_epoch: 0
	checkpoint_year: 2025
	checkpoint_month: 8
	num_cpu_left: 0
	queue_length: 1
	lr: 0.0001
	wd: 1e-05
	batch_size: 8192
	first_n_moves: 10
	last_n_moves: 10
	dim_cnn: 256
	dim_vit: 1024
	num_blocks_cnn: 5
	num_blocks_vit: 2
	input_channels: 18
	vit_length: 8
	elo_dim: 128
	side_info: True
	side_info_coefficient: 1.0
	value: True
	value_coefficient: 1.0
	max_games_per_elo_range: 50000
MAIA2Model(
  (chess_cnn): ChessResNet(
    (conv1): Conv2d(18, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layers): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1

ValueError: num_samples should be a positive integer value, but got num_samples=0

In [ ]:
def from_pretrained(type, device, save_root = "/saves/0.0001_8192_1e-05"):

    if os.path.exists(save_root) == False:
        os.makedirs(save_root)

    if os.path.exists(output_path):
        print(f"Model for {type} games already downloaded.")
    else:
        print(f"Downloading model for {type} games.")
        gdown.download(url, output_path, quiet=False)

    cfg_url = "https://drive.google.com/uc?id=1GQTskYMVMubNwZH2Bi6AmevI15CS6gk0"
    cfg_path = os.path.join(save_root, "config.yaml")
    if not os.path.exists(cfg_path):
        gdown.download(cfg_url, cfg_path, quiet=False)

    cfg = parse_args(cfg_path)

    all_moves = get_all_possible_moves()
    elo_dict = create_elo_dict()

    model = MAIA2Model(len(all_moves), elo_dict, cfg)
    model = nn.DataParallel(model)

    checkpoint = torch.load(output_path, map_location='cpu')
    model.load_state_dict(checkpoint['model_state_dict'])
    model = model.module

    if device == "gpu":
        model = model.cuda()

    print(f"Model for {type} games loaded to {device}.")

    return model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from torch import nn
from maia2.main import MAIA2Model
from maia2.utils import get_all_possible_moves, create_elo_dict, parse_args

def load_my_model(checkpoint_path, config_path, device="cpu"):
    # Parse config file
    cfg = parse_args(config_path)

    # Create model
    all_moves = get_all_possible_moves()
    elo_dict = create_elo_dict()
    model = MAIA2Model(len(all_moves), elo_dict, cfg)
    model = nn.DataParallel(model)  # same as in repo

    # Load checkpoint
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model = model.module  # unwrap DataParallel

    # Send to GPU if needed
    if device == "gpu":
        model = model.cuda()

    print(f"Custom Maia-2 model loaded from {checkpoint_path} to {device}")
    return model

In [ ]:
my_checkpoint = "/content/drive/MyDrive/ChessEngine/epoch_1_2013-01.pgn.pt"
my_config    = "/content/drive/MyDrive/ChessEngine/config.yaml"
my_model = load_my_model(my_checkpoint, my_config, device="cpu")

Custom Maia-2 model loaded from /content/drive/MyDrive/ChessEngine/epoch_1_2013-01.pgn.pt to cpu


In [ ]:
from maia2 import dataset

# Example test dataset included in repo
data = dataset.load_example_test_dataset()

Downloading...
From: https://drive.google.com/uc?id=1fSu4Yp8uYj7xocbHAbjBP6DthsgiJy9X
To: /content/maia2_data/example_test_dataset.csv
100%|██████████| 22.5M/22.5M [00:00<00:00, 204MB/s]


Example test dataset downloaded.


In [ ]:
from maia2 import inference

data, acc = inference.inference_batch(
    data,
    my_model,
    verbose=1,       # progress bar
    batch_size=1024, # adjust depending on GPU memory
    num_workers=4
)

print("Accuracy:", acc)

100%|██████████| 105/105 [39:55<00:00, 22.81s/it]


Accuracy: 0.1061
